# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("cities.csv")

weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Saint-Augustin,98,CA,1601395980,86,51.23,-58.65,64.08,17.72
1,1,Upernavik,84,GL,1601395792,86,72.79,-56.15,24.13,4.88
2,2,Kavieng,29,PG,1601395980,78,-2.57,150.80,80.73,4.27
3,3,Port Hedland,0,AU,1601395981,68,-20.32,118.57,71.60,8.05
4,4,Hilo,1,US,1601395982,64,19.73,-155.09,71.01,5.82
...,...,...,...,...,...,...,...,...,...,...
559,559,Tondano,75,ID,1601396250,94,1.30,124.91,75.20,2.24
560,560,Pandan,100,PH,1601396251,77,14.05,124.17,81.03,5.35
561,561,Ubalá,14,CO,1601396251,63,4.75,-72.54,86.70,5.12
562,562,Batagay,100,RU,1601396252,72,67.63,134.63,38.91,7.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]


In [4]:



# Create  Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
vacation_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

vacation_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,Port Hedland,0,AU,1601395981,68,-20.32,118.57,71.60,8.05
129,129,Shāhābād,0,IN,1601396033,44,27.65,79.95,78.80,4.63
209,209,Vila,0,PT,1601396073,50,42.03,-8.16,75.00,8.99
264,264,Bereda,0,ES,1601396099,35,43.27,-7.54,73.99,3.00
268,268,Werda,0,BW,1601396101,14,-25.27,23.28,72.93,7.72
283,283,Muros,0,ES,1601396111,50,42.78,-9.06,75.00,8.01
311,311,Sinjār,0,IQ,1601396129,23,36.32,41.88,78.42,4.94
340,340,Bela,0,IN,1601396144,58,25.93,81.98,79.84,2.15
427,427,Tabas,0,IR,1601396186,20,33.60,56.92,73.40,4.70
463,463,Pahrump,0,US,1601395923,18,36.21,-115.98,75.00,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = vacation_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
3,Port Hedland,AU,-20.32,118.57,
129,Shāhābād,IN,27.65,79.95,
209,Vila,PT,42.03,-8.16,
264,Bereda,ES,43.27,-7.54,
268,Werda,BW,-25.27,23.28,
283,Muros,ES,42.78,-9.06,
311,Sinjār,IQ,36.32,41.88,
340,Bela,IN,25.93,81.98,
427,Tabas,IR,33.60,56.92,
463,Pahrump,US,36.21,-115.98,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # build url and make API call
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
     # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    

Retrieving Results for Index 3: Port Hedland.
Closest hotel in Port Hedland is The Esplanade Hotel.
Retrieving Results for Index 129: Shāhābād.
Closest hotel in Shāhābād is hotel saif.
Retrieving Results for Index 209: Vila.
Closest hotel in Vila is Hotel Castrum Villae.
Retrieving Results for Index 264: Bereda.
Closest hotel in Bereda is Casa Grande de Anllo.
Retrieving Results for Index 268: Werda.
Missing field/result... skipping.
Retrieving Results for Index 283: Muros.
Closest hotel in Muros is O Prouso.
Retrieving Results for Index 311: Sinjār.
Missing field/result... skipping.
Retrieving Results for Index 340: Bela.
Closest hotel in Bela is OYO 29866 Shivam Palace.
Retrieving Results for Index 427: Tabas.
Closest hotel in Tabas is Amir hotel.
Retrieving Results for Index 463: Pahrump.
Closest hotel in Pahrump is Holiday Inn Express & Suites Pahrump.


In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
3,Port Hedland,AU,-20.32,118.57,The Esplanade Hotel
129,Shāhābād,IN,27.65,79.95,hotel saif
209,Vila,PT,42.03,-8.16,Hotel Castrum Villae
264,Bereda,ES,43.27,-7.54,Casa Grande de Anllo
268,Werda,BW,-25.27,23.28,
283,Muros,ES,42.78,-9.06,O Prouso
311,Sinjār,IQ,36.32,41.88,
340,Bela,IN,25.93,81.98,OYO 29866 Shivam Palace
427,Tabas,IR,33.60,56.92,Amir hotel
463,Pahrump,US,36.21,-115.98,Holiday Inn Express & Suites Pahrump


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info =[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
 

# Add marker layer and info box content on top of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Figure
fig



Figure(layout=FigureLayout(height='420px'))